In [2]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import datetime as dt

# Import data
hsx = pd.read_csv('CafeF.HSX.Upto06.09.2024.csv', parse_dates=['<DTYYYYMMDD>'])
hnx = pd.read_csv('CafeF.HNX.Upto06.09.2024.csv', parse_dates=['<DTYYYYMMDD>'])
upcom = pd.read_csv('CafeF.UpCom.Upto06.09.2024.csv', parse_dates=['<DTYYYYMMDD>'])

# 1Y data
date_filter = hsx['<DTYYYYMMDD>'].unique()[:252]
hsx_252 = hsx[hsx['<DTYYYYMMDD>'].isin(date_filter)]
hnx_252 = hnx[hnx['<DTYYYYMMDD>'].isin(date_filter)]
upcom_252 = upcom[upcom['<DTYYYYMMDD>'].isin(date_filter)]

# Data processing
df_raw = pd.concat([hsx_252, hnx_252, upcom_252])
df_raw.rename(columns = {'<DTYYYYMMDD>': 'Date', '<Ticker>': 'Ticker', '<Close>': 'Close','<Volume>':'Volume', '<Open>': 'Open', '<High>' : 'High', '<Low>' : 'Low'}, inplace=True)
df_raw.sort_values(by = ['Ticker','Date'], ascending=True, inplace=True)
df_raw['Ticker length'] = df_raw['Ticker'].str.len()
df = df_raw[df_raw['Ticker length'] < 4].drop(labels = 'Ticker length', axis = 1)

Bộ lọc các đường MA

In [4]:
def price_over_ma(period):
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df[f'MA{period}'] = ta.sma(df['Close'], length = period)
    df[f'Price higher MA{period}'] = df['Close'] > df[f'MA{period}']
    df_ma200_yesterday = df[(df['Date'] == yesterday) & (df[f'Price higher MA{period}'] == False)][['Ticker']]
    df_ma200_today = df[(df['Date'] == today) & (df[f'Price higher MA{period}'] == True)][['Ticker']]
    df_ma200_signal = df_ma200_today.merge(df_ma200_yesterday, on = 'Ticker')
    return df_ma200_signal['Ticker'].unique()

def price_under_ma(period):
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df[f'MA{period}'] = ta.sma(df['Close'], length = period)
    df[f'Price under MA{period}'] = df['Close'] < df[f'MA{period}']
    df_ma200_yesterday = df[(df['Date'] == yesterday) & (df[f'Price under MA{period}'] == False)][['Ticker']]
    df_ma200_today = df[(df['Date'] == today) & (df[f'Price under MA{period}'] == True)][['Ticker']]
    df_ma200_signal = df_ma200_today.merge(df_ma200_yesterday, on = 'Ticker')
    return df_ma200_signal['Ticker'].unique()

def ma_convergence(shortMA, mediumMA, longMA):
    df[f'MA{shortMA}'] = ta.sma(df['Close'], length = shortMA)
    df[f'MA{mediumMA}'] = ta.sma(df['Close'], length = mediumMA)
    df[f'MA{longMA}'] = ta.sma(df['Close'], length = longMA)
    df[f'MA{shortMA} higher than MA{mediumMA}'] = df[f'MA{shortMA}'] > df[f'MA{mediumMA}']
    df[f'MA{shortMA} higher than MA{longMA}'] = df[f'MA{shortMA}'] > df[f'MA{longMA}']
    df[f'MA{mediumMA} higher than MA{longMA}'] = df[f'MA{mediumMA}'] > df[f'MA{longMA}']
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df_ma_condition = df[(df['Date'] == today) & (df[f'MA{shortMA}'] > df[f'MA{mediumMA}']) & (df[f'MA{shortMA}'] > df[f'MA{longMA}'])]
    #Signal
    df_ma_yesterday = df[(df['Date'] == yesterday) & (df[f'MA{mediumMA} higher than MA{longMA}'] == False)][['Ticker','MA20 higher than MA50']]
    df_ma_today = df[(df['Date'] == today) & (df[f'MA{mediumMA} higher than MA{longMA}'] == True)][['Ticker']]
    df_signal = df_ma_today.merge(df_ma_yesterday, on = 'Ticker')
    ma_intersection = df_ma_condition.merge(df_signal, on = 'Ticker')
    return ma_intersection['Ticker'].unique()

Bộ lọc Bollinger Bands

In [ ]:
##### Giá đang nằm ở ngoài 2 biên Bollinger Bands
#Giá 'Close' đang nằm ở ngoài biên trên 2 ngày liên tục
def price_out_of_upper(period, sd):
    df['Upper'] = ta.bbands(df['Close'], length = period, std = sd).iloc[:,2]
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df['Close price over Upper'] = df['Close'] > df['Upper']
    closeprice_upper_yesterday = df[(df['Date'] == yesterday) & (df['Close price over Upper'] == True)][['Ticker']]
    closeprice_upper_today = df[(df['Date'] == today) & (df['Close price over Upper'] == True)][['Ticker']]
    price_over_upper = closeprice_upper_today.merge(closeprice_upper_yesterday, on = 'Ticker')
    return price_over_upper['Ticker'].unique()

#Giá 'Close' đang nằm ở ngoài biên dưới 2 ngày liên tục
def price_out_of_lower(period, sd):
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df['Lower'] = ta.bbands(df['Close'], length = period, std = sd).iloc[:,0]
    df['Close price under Lower'] = df['Close'] < df['Lower']
    closeprice_lower_yesterday = df[(df['Date'] == yesterday) & (df['Close price under Lower'] == True)][['Ticker']]
    closeprice_lower_today = df[(df['Date'] == today) & (df['Close price under Lower'] == True)][['Ticker']]
    price_over_lower = closeprice_lower_today.merge(closeprice_lower_yesterday, on = 'Ticker')
    return price_over_lower['Ticker'].unique()


#####Giá thoát ra ngoài 2 biên
#Giá thoát ra ngoài biên trên
def price_move_over_upper(period, sd):
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]   
    df['Upper'] = ta.bbands(df['Close'], length = period, std = sd).iloc[:,2]
    # Conditions
    df['Low price under Upper'] = df['Low'] < df['Upper']
    df['Close price over Upper'] = df['Close'] > df['Upper']
    #Giá thoát ra ngoài biên trên
    price_moving_over_upper = df[(df['Date'] == today) & (df['Low price under Upper'] == True) & (df['Close price over Upper'] == True)][['Ticker']]
    return price_moving_over_upper['Ticker'].unique()

#Giá thoát ra ngoài biên dưới
def price_move_over_lower(period, sd):
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df['Lower'] = ta.bbands(df['Close'], length = period, std = sd).iloc[:,0]
    #Conditions
    df['High price over Lower'] = df['High'] > df['Lower']
    df['Close price under Lower'] = df['Close'] < df['Lower']
    price_moving_over_lower = df[(df['Date'] == today) & (df['High price over Lower'] == True) & (df['Close price under Lower'] == True)][['Ticker']]
    return price_moving_over_lower['Ticker'].unique()


#####Giá đi vào từ ngoài 2 biên  
#Giá cắt xuống từ ngoài biên trên
def price_move_into_upper(period, sd):
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df['Upper'] = ta.bbands(df['Close'], length = period, std = sd).iloc[:,2]
    #Conditions
    df['High price over Upper'] = df['High'] > df['Upper']
    df['Close price under Upper'] = df['Close'] < df['Upper']
    price_cut_under_upper = df[(df['Date'] == today) & (df['High price over Upper'] == True) & (df['Close price under Upper'] == True)][['Ticker']]
    return price_cut_under_upper['Ticker'].unique()


#Giá cắt lên từ ngoài biên dưới
def price_move_into_lower(period, sd):
    today = df['Date'].iloc[-1]
    yesterday = df['Date'].iloc[-2]
    df['Lower'] = ta.bbands(df['Close'], length = period, std = sd).iloc[:,0]
    #Conditions
    df['Low price under Lower'] = df['Low'] < df['Lower']
    df['Close price over Lower'] = df['Close'] > df['Lower']
    price_cut_over_lower = df[(df['Date'] == today) & (df['Low price under Lower'] == True) & (df['Close price over Lower'] == True)][['Ticker']]
    return price_cut_over_lower['Ticker'].unique()